In [1]:
#import dependencies
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import numpy as np
import datetime 
import psycopg2 as pg
import pandas.io.sql as psql
import matplotlib as plt

# *E* extract data sets

In [2]:
#connect and load csv file
amzn_file = "Asset_Classes/AMZN_Historical.csv"
bitcoin_file = "Resources/clean_bitcoin.csv"
gld_file = "Asset_Classes/GLD_Historical.csv"
nflx_file = "Asset_Classes/NFLX_Historical.csv"
shy_file = "Asset_Classes/SHY_Historical.csv"
sp_file = "Asset_Classes/SP_Historical.csv"
uso_file = "Asset_Classes/USO_Historical.csv"
zm_file = "Asset_Classes/ZM_Historical.csv"

In [6]:
#load into df
amzn_df = pd.read_csv(amzn_file)
bitcoin_df  = pd.read_csv(bitcoin_file)
gld_df = pd.read_csv(gld_file)
nflx_df = pd.read_csv(nflx_file)
shy_df = pd.read_csv(shy_file)
sp_df = pd.read_csv(sp_file)
uso_df = pd.read_csv(uso_file)
zm_df= pd.read_csv(zm_file)

# *T* transform bitcoin_Df

In [8]:
#transform bitcoin_Df
bitcoin_df[["date"]] = bitcoin_df[["date"]].apply(pd.to_datetime)
#drop unnamed:0 column
new_bitcoin_df = bitcoin_df.drop(columns=['Unnamed: 0'])
#rename column to match sql database table
new_bitcoin_df.rename(columns={'close': 'btc_close'}, inplace=True)
#drop duplicates
new_bitcoin_data = new_bitcoin_df.drop_duplicates(subset='date', keep='last', inplace=False)
new_bitcoin_data.head()

,date,btc_close
1397,2020-01-04,7344.48
2808,2020-01-05,7356.70
4237,2020-01-06,7762.74
5676,2020-01-07,8159.01
7114,2020-01-08,8044.44


In [10]:
amzn_df.rename(columns={'close': 'amzn_close'}, inplace=True)
gld_df.rename(columns={'close': 'gld_close'}, inplace=True)
nflx_df.rename(columns={'close': 'nflx_close'}, inplace=True)
shy_df.rename(columns={'close': 'shy_close'}, inplace=True)
sp_df.rename(columns={'close': 'sp_close'}, inplace=True)
uso_df.rename(columns={'close': 'uso_close'}, inplace=True)
zm_df.rename(columns={'close': 'zm_close'}, inplace=True)

,date,open,high,low,zm_close,volume
0,12/30/2019,66.559998,67.180000,65.519997,66.790001,1162700
1,12/31/2019,66.379997,68.180000,66.309998,68.040001,1351400
2,1/2/2020,68.800003,69.235001,68.019997,68.720001,1315500
3,1/3/2020,67.620003,68.680000,67.099998,67.279999,1127900
4,1/6/2020,66.629997,70.495003,65.811996,70.320000,3151600


In [17]:
new_amzn_data = amzn_df[["date","amzn_close"]]
new_gld_data = gld_df[["date","gld_close"]]
new_nflx_data = nflx_df[["date","nflx_close"]]
new_shy_data = shy_df[["date","shy_close"]]
new_sp_data = sp_df[["date","sp_close"]]
new_uso_data = uso_df[["date","uso_close"]]
new_zm_data = zm_df[["date","zm_close"]]

In [20]:
new_amzn_data.dtypes

date           object
amzn_close    float64
dtype: object

In [23]:
#convert "date" column from object to datetime type
new_amzn_data[["date"]] = new_amzn_data[["date"]].apply(pd.to_datetime)
new_gld_data[["date"]] = new_gld_data[["date"]].apply(pd.to_datetime)
new_nflx_data[["date"]] = new_nflx_data[["date"]].apply(pd.to_datetime)
new_shy_data[["date"]] = new_shy_data[["date"]].apply(pd.to_datetime)
new_sp_data[["date"]] = new_sp_data[["date"]].apply(pd.to_datetime)
new_uso_data[["date"]] = new_uso_data[["date"]].apply(pd.to_datetime)
new_zm_data[["date"]] = new_zm_data[["date"]].apply(pd.to_datetime)

# Merge data sets

In [33]:
one = pd.merge(new_amzn_data,new_bitcoin_data[['date','btc_close']],
                        on = 'date')

In [34]:
two = pd.merge(one,new_gld_data [['date','gld_close']],
                        on = 'date')

In [35]:
three = pd.merge(two,new_nflx_data [['date','nflx_close']],
                        on = 'date')

In [36]:
four = pd.merge(three,new_shy_data[['date','shy_close']],
                        on = 'date')

In [37]:
five = pd.merge(four,new_sp_data[['date','sp_close']],
                        on = 'date')

In [38]:
six = pd.merge(five,new_uso_data [['date','uso_close']],
                        on = 'date')

In [39]:
seven = pd.merge(six,new_zm_data [['date','zm_close']],
                        on = 'date')

In [40]:
seven.head()

,date,amzn_close,btc_close,gld_close,nflx_close,shy_close,sp_close,uso_close,zm_close
0,2019-12-30,1846.890015,NaN,142.630005,323.309998,84.639999,3223.5,12.89,66.790001
1,2019-12-31,1847.839966,NaN,142.899994,323.570007,84.629997,3231.0,12.81,68.040001
2,2020-01-02,1898.010010,NaN,143.949997,329.809998,84.669998,3259.0,12.81,68.720001
3,2020-01-03,1874.969971,NaN,145.860001,325.899994,84.739998,3235.5,13.18,67.279999
4,2020-01-06,1902.880005,7762.74,147.389999,335.829987,84.709999,3243.5,13.16,70.320000


In [41]:
#drop nan rows
seven.dropna()

,date,amzn_close,btc_close,gld_close,nflx_close,shy_close,sp_close,uso_close,zm_close
4,2020-01-06,1902.880005,7762.74,147.389999,335.829987,84.709999,3243.50,13.16,70.320000
5,2020-01-07,1906.859985,8159.01,147.970001,330.750000,84.709999,3235.25,13.13,71.900002
6,2020-01-08,1891.969971,8044.44,146.860001,339.260010,84.669998,3260.25,12.66,72.550003
7,2020-01-09,1901.050049,7806.78,146.029999,335.660004,84.680000,3276.00,12.49,72.620003
8,2020-01-10,1883.160034,8200.00,146.910004,329.049988,84.699997,3264.75,12.41,73.089996
...,...,...,...,...,...,...,...,...,...
74,2020-04-16,2408.189941,7111.67,161.710007,439.170013,86.680000,2863.25,4.36,150.259995
75,2020-04-17,2375.000000,7024.19,158.570007,422.959991,86.650002,2869.00,4.21,150.059998
76,2020-04-20,2393.610107,6841.00,159.699997,437.489990,86.650002,2814.00,3.75,148.990005
77,2020-04-21,2328.120117,6854.98,158.610001,433.829987,86.669998,2736.50,2.81,143.270004
